In [ ]:
!conda install pytorch torchvision torchaudio cudatoolkit=10.2 -c pytorch
!conda install -c conda-forge opencv
!conda install -c anaconda pywin32
!conda install -c anaconda pillow
!conda install -c conda-forge kornia

!pip install d3dshot

In [1]:
from pathlib import Path
from math import log10
import argparse
from pathlib import Path
import random
import time
import os
import subprocess
import subprocess
import threading
import queue

import torch
import torch.utils.data as data
from torch import nn
from torch.nn.functional import relu
from torchvision import transforms
from torchvision.transforms import ToTensor, RandomCrop
from torch import nn
from torch import optim
from torch.utils.data import DataLoader
from torch.autograd import Variable
import torchvision 

import cv2
import numpy as np
import kornia
import d3dshot
import win32gui, win32ui, win32con, win32api

torch.backends.cudnn.benchmark = True
dev = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(dev)

class SRCNN(nn.Module):
    def __init__(self):
        super(SRCNN, self).__init__()

        self.conv0   = nn.Conv2d(1, 16, 3, padding=1, padding_mode='replicate' )

        self.LR1_1   = nn.LeakyReLU()
        self.conv1_1 = nn.Conv2d(16, 16, 3, padding=1, padding_mode='replicate')
        self.LR1_2   = nn.LeakyReLU()
        self.DO1     = nn.Dropout(0.3)
        self.conv1_2 = nn.Conv2d(16, 16, 3, padding=1, padding_mode='replicate')


        self.LR2_1   = nn.LeakyReLU()
        self.conv2_1 = nn.Conv2d(16, 16, 3, padding=1, padding_mode='replicate')
        self.LR2_2   = nn.LeakyReLU()
        self.DO2     = nn.Dropout(0.3)
        self.conv2_2 = nn.Conv2d(16, 16, 3, padding=1, padding_mode='replicate')
        
        
        self.LR3_1   = nn.LeakyReLU()
        self.conv3_1 = nn.Conv2d(16, 16, 3, padding=1, padding_mode='replicate')
        self.LR3_2   = nn.LeakyReLU()
        self.DO3     = nn.Dropout(0.3)
        self.conv3_2 = nn.Conv2d(16, 16, 3, padding=1, padding_mode='replicate')
        

        self.convL   = nn.Conv2d(16, 1, 3, padding=1, padding_mode='replicate')
    def forward(self, x):

        A0 = self.conv0(x)

        x = self.LR1_1(A0)      
        x = self.conv1_1(x)
        x = self.LR1_2(x)
        x = self.DO1(x)
        x = self.conv1_2(x)
        x=x+A0
        A1=x

        x = self.LR2_1(x)      
        x = self.conv2_1(x)
        x = self.LR2_2(x)
        x = self.DO2(x)
        x = self.conv2_2(x)
        x=x+A0+A1
        A2=x        
        
        x = self.LR3_1(x)      
        x = self.conv3_1(x)
        x = self.LR3_2(x)
        x = self.DO3(x)
        x = self.conv3_2(x)
        x=x+A0+A1+A2

        x = self.convL(x)
        return x

model = SRCNN()
model_path = './W/r3c1x240_D+vgg.pth'
model.load_state_dict(torch.load(model_path, map_location=torch.device(dev)))
model=model.to(dev).half()
model=torch.jit.script(model)



d = d3dshot.create(capture_output="numpy")
d.display = d.displays[1]
fps=30

fB = queue.Queue(maxsize=3)



def ss():
    while True: 
        start_time = time.perf_counter()
        
        img=d.screenshot()   
        frame = img
        fB.put(frame)
        
        x1=time.perf_counter() - start_time 
        start_timeD = time.perf_counter()
        print(x1)
        while (time.perf_counter() - start_timeD) < (1/fps) - x1 :
            pass
    
def Pred():
       
  cv2.namedWindow("cnn")  
  a = win32gui.FindWindow(None,"cnn")
  win32gui.SetWindowLong(a, win32con.GWL_STYLE, win32con.WS_POPUP)
  x = 0
  y = 0
  width = 1920
  height = 1080 
    
    
  while True:
    
    start_time = time.perf_counter()
    
    
    img=fB.get(timeout=2)
    frame = img
    frame= torch.from_numpy(frame).to(dev).reshape([1, 1080, 1920, 3]).permute(0, 3, 1, 2).float()#nchw
    
    frameYUV= kornia.rgb_to_yuv(frame)
    frameY  = frameYUV[:,0,:,:].reshape([1, 1, 1080, 1920]).half()
    predY=model(frameY/255)*255 

    f0=predY.float()[0,0,:,:]
    f1=frameYUV[0,1,:,:]
    f2=frameYUV[0,2,:,:] #1080,1920
    pred=torch.stack([f0,f1,f2]).float() # 3,1080,1920
    pred=kornia.yuv_to_rgb(pred).clip(0,255).permute(1,2,0)[:,:,[2, 1, 0]]
    pred=torch.tensor(pred,dtype=torch.uint8).to('cpu').numpy()

    cv2.imshow('cnn', pred)
    win32gui.SetWindowPos(a, win32con.HWND_TOPMOST, x, y, width, height, win32con.SWP_SHOWWINDOW)
    if cv2.waitKey(1) & 0xFF == ord('q'):
     break  
    
    x1=time.perf_counter() - start_time 
    start_timeD = time.perf_counter()



T_SS = threading.Thread(target=ss)
T_Pred = threading.Thread(target=Pred)

T_SS.start()
T_Pred.start()

cuda:0
0.013643300000000025
0.021839299999999895
0.019656599999999358
0.035038200000000685


C:\Users\Intel810\.conda\envs\opencv\lib\site-packages\ipykernel_launcher.py:152: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


1.6434300999999998
0.11656250000000057
0.04520169999999979
0.04551479999999941
0.045225300000000246
0.04313969999999934
0.045562000000000324
0.0442657999999998
0.043302800000001085
0.047014299999998954
0.04492079999999987
0.04511529999999908
0.04464019999999991
0.044616200000000106
0.04520969999999913
0.04775699999999894
0.043983100000000164
0.04626230000000042
0.047052199999999544
0.04483679999999879
0.04516059999999911
0.045835400000001414
0.044861700000000226
0.06121360000000031
0.044360700000000364
0.04441270000000053
0.04450360000000053
0.04387720000000073
0.04588940000000008
0.04459869999999988
0.04439369999999698
0.04429389999999955
0.04494629999999944
0.04513979999999762
0.04444050000000033
0.04398079999999993
0.04318609999999978
0.046329400000001186
0.0458469000000008
0.04456289999999896
0.04429630000000273
0.043213600000001406
0.04699529999999896
0.04528009999999938
0.045624200000002446
0.050440900000001676
0.0455091000000003
0.042989299999998565
0.044791899999999885
0.044471

0.04441910000000604
0.04356059999999928
0.04344379999999859
0.04338860000000011
0.04339679999999646
0.04517859999999985
0.0439718999999954
0.04475659999999948
0.04453249999999542
0.04491519999999838
0.04614029999999758
0.04576630000000392
0.045142800000000705
0.04446829999999835
0.04409429999999759
0.04439149999999614
0.044576199999994515
0.044207600000000014
0.04424870000000425
0.04894539999999381
0.04545070000000351
0.04456980000000499
0.04476969999999625
0.0439109000000002
0.04510419999999726
0.04466040000000504
0.04552149999999955
0.04371030000000076
0.042595499999997344
0.04467630000000611
0.04426890000000583
0.044025300000001266
0.04654899999999884
0.04470709999999656
0.044817599999994684
0.04549979999999465
0.045213000000003944
0.04560159999999769
0.043648499999996204
0.04371510000000001
0.043953500000000645
0.04768190000000061
0.04594639999999828
0.04436450000000036
0.04398590000000269
0.04609320000000139
0.04461400000000282
0.044755700000003173
0.04480800000000329
0.0450519999

0.04347470000000442
0.04400590000000193
0.04368199999999689
0.04474239999999696
0.04391509999999954
0.04356240000000611
0.04474609999999757
0.04465489999999761
0.04504160000000468
0.04914099999999877
0.0471798000000021
0.04373389999999944
0.04327870000000189
0.04328729999999581
0.04458940000000666
0.0455452000000065
0.04475630000000308
0.04444300000000112
0.04450429999999983
0.04539129999999858
0.044954400000001726
0.04479919999999993
0.0461925999999977
0.04446860000000186
0.043588200000002075
0.04347859999999315
0.044779099999999517
0.043919000000002484
0.043736499999994294
0.044506300000001886
0.04540190000000166
0.04479870000000119
0.045141100000002155
0.04471589999999992
0.044555599999995366
0.045230799999998794
0.04485680000000514
0.04463609999999818
0.04468010000000078
0.043771800000001804
0.04339669999999529
0.04354060000000004
0.04498040000000003
0.04474929999999944
0.043734299999997006
0.04470519999999567
0.04462140000000403
0.045145800000000236
0.044886099999999374
0.04516319

...

0.04574760000002698
0.04464550000000145
0.04452179999998407
0.04422069999998257
0.04338569999998754
0.043689699999958975
0.04428079999996726
0.04352730000005067
0.04472470000001749
0.04485150000004978
0.05062900000001491
0.04629999999997381
0.04529320000000325
0.04465699999997241
0.04391409999999496
0.04417839999996431
0.04356749999999465
0.04438329999999269
0.043409300000007534
0.04337550000002466
0.04491560000002437
0.04501679999998487
0.04407739999999194
0.043746100000021215
0.044560399999966194
0.0446034999999938
0.0448221999999987
0.04521149999999352
0.04475999999999658
0.04478490000002466
0.04426829999999882
0.04523599999998851
0.04344059999999672
0.04383109999997714
0.04363079999995989
0.04504229999997733
0.04506259999999429
0.04465240000001813
0.0450645000000236
0.044811500000037086
0.045906099999967864
0.044903900000008434
0.0439469000000372
0.04403469999999743
0.04593070000004218
0.04422180000000253
0.04344770000000153
0.04347020000000157
0.044806499999992866
0.04443040000001

0.04441449999995939
0.044662999999957265
0.04610120000000961
0.044421399999976074
0.04496050000000196
0.04483650000003081
0.04504739999998719
0.04504140000000234
0.04436640000000125
0.04361779999999271
0.04499759999998787
0.046092799999996714
0.0434223999999972
0.045418100000006234
0.04455089999999018
0.0454244000000017
0.04466650000000527
0.04443049999997584
0.04506420000001299
0.04501679999998487
0.04456050000004552
0.04547780000001467
0.04477830000001859
0.04370579999999791
0.04339229999999361
0.04936559999998735
0.0443576000000121
0.04472349999997505
0.044917199999986224
0.044931899999994585
0.0459301999999866
0.04566330000000107
0.044382900000016434
0.04451800000003914
0.043171799999981886
0.044370500000013635
0.04643859999998767
0.04363260000002356
0.044561799999996765
0.04478510000001279
0.045206800000016756
0.04495370000000776
0.04484109999998509
0.04464679999995269
0.04486470000000509
0.0455499000000259
0.0448385999999914
0.048915800000031595
0.045547800000008465
0.04396669999

0.04437889999996969
0.04522989999998117
0.04475409999997737
0.044802500000002965
0.044855699999970966
0.044987300000002506
0.04556820000004791
0.04557300000004716
0.04463820000000851
0.04403239999999187
0.0455387000000087
0.04389739999999165
0.04434609999998429
0.04345130000001518
0.04380960000003142
0.044961000000000695
0.04814770000001545
0.04536660000002257
0.044981899999982033
0.04444940000001907
0.045410500000002685
0.044664399999987836
0.04333970000004683
0.04349960000001829
0.04366459999999961
0.043846400000006724
0.04436989999999241
0.04408749999998918
0.04462130000001707
0.044577399999980116
0.04502270000000408
0.044875999999987926
0.044741499999986445
0.044971100000054776
0.044623900000033245
0.045212700000035966
0.043566999999995915
0.043654099999969276
0.04356489999997848
0.044029299999976956
0.04394750000000158
0.04483140000002095
0.04356629999995221
0.04475400000001173
0.04483759999999393
0.045226599999978134
0.044772800000032476
0.044761899999969046
0.04564519999996719
0

0.04551170000002003
0.04442740000001777
0.04465340000001561
0.04451619999997547
0.044660399999997935
0.04511829999995598
0.04449440000001914
0.04476110000001654
0.04509030000002667
0.04541010000002643
0.04492590000000973
0.0425387000000228
0.04704559999998992
0.04635540000003857
0.0450017999999659
0.044420500000001084
0.04609640000001036
0.04479070000002139
0.045022300000027826
0.04458230000000185
0.04546040000002449
0.04431849999997439
0.044818800000030024
0.0438581999999883
0.04388000000000147
0.04414099999996779
0.04571319999996604
0.04481650000002446
0.04469360000001643
0.04484029999997574
0.04552419999998847
0.04530579999999418
0.044935800000018844
0.0448718000000099
0.044452499999977135
0.04493379999996705
0.043422099999986585
0.04468500000001541
0.04349480000001904
0.04369399999995949
0.04471669999998085
0.046227899999962574
0.04469779999999446
0.044673100000011345
0.04495349999996279
0.044848000000001775
0.045758500000033564
0.04497559999998657
0.04470129999998562
0.04467139999

0.04484079999997448
0.04479300000002695
0.04498200000000452
0.04479520000001003
0.044473100000004706
0.04967460000000301
0.04634439999995266
0.04481649999996762
0.042763799999988805
0.04465290000001687
0.04555620000002136
0.04331809999996494
0.04339429999998856
0.04474920000001248
0.04480799999998908
0.04464269999999715
0.044761800000003404
0.04473950000004834
0.04529030000003331
0.04499359999999797
0.045124899999962054
0.044578500000000076
0.0441361000000029
0.04402879999997822
0.04435280000001285
0.04366509999999835
0.04371570000000702
0.04339470000002166
0.0446407000000022
0.04510279999999511
0.04510629999998628
0.04480839999996533
0.04474270000002889
0.04478209999996352
0.04493289999999206
0.04515890000004674
0.04401319999999487
0.04422319999997626
0.04361120000004348
0.0446142999999779
0.04480209999996987
0.04418240000001106
0.04469530000000077
0.04485060000001795
0.045475500000009106
0.04459810000003017
0.04473489999998037
0.045056499999986954
0.04506459999998924
0.04441140000000